# Alice CID categorization sheet export

The *gestão de saúde magenta* squad inputs dw weights for daly calculations manually in a google sheet. This notebook is to export the sheet, after downloading from the Google Drive and storing it in a local computer. Since the sheet is updated not very often, there is not much gain from automating the process in Airflow (*yet*).

Steps:

    - Download the google sheet DALY weights and store it in the 'redshift_import_export\external\' package folder;

    - run this notebook to upload to Redshift;
    
    - use the DW to make calculations, new tables, dashes...
***

## 0. Setup

In [15]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [16]:
alice_categories = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/alice_cid_categories.csv')
overall_cid_categories = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/overall_cid_categories.csv')
alice_acute_classes = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/alice_acute_class.csv')

### 2. Transform

In [17]:
overall_cid_categories.rename(columns={'Disease.Code.Value': 'cid','capitulo': 'chapter', 'grupos': 'group', 'categorias': 'category', 'subcategorias': 'subcategory'}, inplace = True)
overall_cid_categories = overall_cid_categories[['cid', 'chapter', 'group', 'category', 'subcategory']]
print(overall_cid_categories.columns)


Index(['cid', 'chapter', 'group', 'category', 'subcategory'], dtype='object')


In [18]:
alice_categories.rename(columns={'Disease.Code.Value': 'cid', 'Categoria 2.0': 'alice_category'}, inplace = True)
print(alice_categories.columns)


Index(['cid', 'alice_category'], dtype='object')


In [19]:
alice_acute_classes.rename(columns = {'CID 10': 'cid','categoria alice': 'alice_category', 'agudo ou longitudinal?': 'alice_acute_class'}, inplace = True)
alice_acute_classes = alice_acute_classes[['cid', 'alice_acute_class']]
alice_acute_classes = alice_acute_classes.drop_duplicates()
print(alice_acute_classes.columns)

Index(['cid', 'alice_acute_class'], dtype='object')


## 3. Merge

In [21]:
df = overall_cid_categories.merge(alice_categories, how = 'left', on = 'cid').merge(alice_acute_classes, how = 'left', on = 'cid')
df.drop_duplicates(subset=['cid'], inplace = True)
df.head(10)

,cid,chapter,group,category,subcategory,alice_category,alice_acute_class
0,A01,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Febres tifóide e paratifóide,Febres tifóide e paratifóide,NaN,agudo
1,A02,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Outras infecções por Salmonella,Infecções gastrointestinais,NaN
2,A022,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções por Salmonella,Infecções localizadas por salmonela,NaN,agudo
3,A04,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Outras infecções intestinais bacterianas,Infecções gastrointestinais,NaN
4,A049,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras infecções intestinais bacterianas,Infecção intestinal bacteriana não especificada,Infecções gastrointestinais,agudo
5,A05,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,"Outras intoxicações alimentares bacterianas, n...","Outras intoxicações alimentares bacterianas, n...",NaN,NaN
6,A059,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,"Outras intoxicações alimentares bacterianas, n...",Intoxicação alimentar bacteriana não especificada,Infecções gastrointestinais,agudo
7,A06,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Amebíase,Amebíase,Infecções gastrointestinais,NaN
8,A069,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Amebíase,Amebíase não especificada,NaN,agudo
9,A071,Capítulo I - Algumas doenças infecciosas e par...,Doenças infecciosas intestinais,Outras doenças intestinais por protozoários,Giardíase [lamblíase],Infecções gastrointestinais,agudo


## 4. Export

In [22]:
di.redshift.insert_table(df, table_name='alice_cid_categories', if_exists='replace')

Table alice_cid_categories inserted successfully, with 2907 rows.
